<a href="https://colab.research.google.com/github/chihchao/AoP/blob/main/crawler_api_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 高雄紅橘線捷運車站中心座標

dataset:
https://data.gov.tw/dataset/102187

data url:
https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460/resource/662f822c-632b-4cb3-80f6-b5ac053d210b/download/11311.json

In [1]:
import requests


url = "https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460/resource/662f822c-632b-4cb3-80f6-b5ac053d210b/download/11311.json"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
data[0]

{'seq': 1,
 '車站編號': 'R3',
 '車站中文名稱': '小港',
 '車站英文名稱': 'Siaogang',
 'Latitude': 22.56481191,
 'Longitude': 120.3538521}

In [ ]:
keys = data[0].keys()
keys

dict_keys(['seq', '車站編號', '車站中文名稱', '車站英文名稱', 'Latitude', 'Longitude'])

In [ ]:
with open('station.csv', 'w', encoding='utf-8') as f:
    header = ','.join(keys)
    f.write(header + '\n')
    for row in data:
        values = [str(row.get(key, '')) for key in keys]
        values = ','.join(values)
        f.write(values + '\n')

In [ ]:
from os import write
import csv
with open('station.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(data)

## Make an APP

In [ ]:
!pip install gradio

In [14]:
import requests
import gradio as gr

def get_data(url):
    response = requests.get(url)
    data = response.json()
    return data

def get_data_ids(data, key):
    return [ str(row.get(key, '')) for row in data]

def get_station_info(station_id):
    data = stations.copy()
    station = next((row for row in data if row['車站編號'] == station_id), None)
    if station:
        intr = f'車站編號：{station["車站編號"]}\n'
        intr += f'車站中文名稱：{station["車站中文名稱"]}\n'
        intr += f'車站英文名稱：{station["車站英文名稱"]}\n'
        return intr
    else:
        return "No data found."

url = 'https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460/resource/662f822c-632b-4cb3-80f6-b5ac053d210b/download/11311.json'


stations = get_data(url)
station_ids = get_data_ids(stations, '車站編號')

dropdown = gr.Dropdown(choices=station_ids, label="請選擇一個車站編號：")
output = gr.Textbox(label="車站資訊：")

gr.Interface(fn=get_station_info, inputs=dropdown, outputs=output).launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://818cc1c4901cd53e8f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 加上地圖

In [17]:
import requests
import gradio as gr

def get_data(url):
    response = requests.get(url)
    data = response.json()
    return data

def get_data_ids(data, key):
    return [ str(row.get(key, '')) for row in data]

def get_station_info(station_id):
    data = stations.copy()
    station = next((row for row in data if row['車站編號'] == station_id), None)
    if station:
        intr = f'車站編號：{station["車站編號"]}\n'
        intr += f'車站中文名稱：{station["車站中文名稱"]}\n'
        intr += f'車站英文名稱：{station["車站英文名稱"]}\n'
        map = generate_map({'lat': float(station["Latitude"]), 'lon': float(station["Longitude"])})
        return intr, map
    else:
        return "No data found."

def generate_map(location):
    map_html = f"""
    <div style="width: 100%; height: 400px;">
        <iframe
            width="100%"
            height="100%"
            frameborder="0"
            scrolling="no"
            marginheight="0"
            marginwidth="0"
            src="https://www.openstreetmap.org/export/embed.html?bbox={location['lon']-0.01}%2C{location['lat']-0.01}%2C{location['lon']+0.01}%2C{location['lat']+0.01}&layer=mapnik&marker={location['lat']}%2C{location['lon']}">
        </iframe>
    </div>
    """
    return map_html


url = 'https://data.kcg.gov.tw/dataset/2629db6d-98ec-4b24-8ab1-5dc34d998460/resource/662f822c-632b-4cb3-80f6-b5ac053d210b/download/11311.json'


stations = get_data(url)
station_ids = get_data_ids(stations, '車站編號')

dropdown = gr.Dropdown(choices=station_ids, label="請選擇一個車站編號：")
output_text = gr.Textbox(label="Post Details")
output_map = gr.HTML(label="Location")

# 配置 Gradio 界面
gr.Interface(
    fn=get_station_info,
    inputs=dropdown,
    outputs=[output_text, output_map]
).launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9034e5a6f22a63450b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
